In [1]:
%pip install beautifulsoup4 lxml html5lib
%pip install EbookLib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from zipfile import ZipFile
import shutil
import glob



In [2]:
import warnings
from bs4 import MarkupResemblesLocatorWarning, XMLParsedAsHTMLWarning, BeautifulSoup
import bs4
#warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

In [3]:
def trimEnd(s: str, value: str):
  if(s.endswith(value)):
    length = len(value)
    return s[:-length]
  return s

In [4]:


def extractEpub(fromFile: str, toDir: str):
    shutil.unpack_archive(f'input/{fromFile}', f'data/{toDir}', 'zip')


def saveEpub(fromDir: str, toFile: str):
    shutil.make_archive(f'output/{toFile}', 'zip', f'data/{fromDir}')

In [5]:

inputEpubs = glob.glob('*.epub', root_dir='./input')

In [6]:
def readAll(filename: str):
  file = open(filename, encoding='utf-8')
  text = file.read()
  file.close()
  return text

def writeAll(filename: str, text: str):
  file = open(filename, encoding='utf-8', mode='w')
  file.write(text)
  file.close()


In [7]:
from typing import List

from bs4 import NavigableString, Tag

def parseText(soup: BeautifulSoup, text: str):
    def embolden(t: str):
        dividerIndex = len(t) // 4 + 1
        boldPart = soup.new_tag('b')
        boldPart.string = t[0: dividerIndex]

        return [boldPart, soup.new_string(t[dividerIndex:])]

    result: List[Tag | NavigableString] = []

    startI = 0
    while startI < len(text):
        char = text[startI]
        if not char.isalpha():
            result.append(char)
            startI += 1
            continue

        if char == '\'' or char == '"':
            result.extend(embolden(char))
            startI += 1
            continue

        endI = startI
        while endI < len(text) and text[endI].isalpha():
            endI += 1

        word = text[startI: endI]

        result.extend(embolden(word))
        startI = endI
    return result


In [8]:
import os
for epubFullFileName in inputEpubs:
  epubShortFileName = trimEnd(epubFullFileName, '.epub')
  try:
    shutil.rmtree(f'./data/{epubShortFileName}')
    os.remove(f'./output/{epubFullFileName}')
  except FileNotFoundError:
    pass
  finally:
    pass
for dir in glob.glob('./data/*'):
    shutil.rmtree(dir)
for file in glob.glob('./output/*'):
    os.remove(file)

In [9]:
from typing import Iterable, Iterator

from bs4 import PageElement

def skip(iter: Iterator, count: int):
  for i in range(0, count):
    try:
      iter.__next__()
    except StopIteration:
      return

def parseTree(soup: BeautifulSoup, start: BeautifulSoup, elements: Iterable[PageElement]):
  iter = elements.__iter__()
  while True:
    try:
      next = iter.__next__()

      if isinstance(next, Tag):
        parseTree(soup, start, next.children)
      else:
        boldText = parseText(soup, next.string)
        next.replace_with(*boldText)
        skip(iter, len(boldText) - 1)

    except StopIteration:
      break





In [10]:
import os

for epubFullFileName in inputEpubs:
    print(epubFullFileName)
    epubShortFileName = trimEnd(epubFullFileName, '.epub')

    extractEpub(epubFullFileName, epubShortFileName)
    htmlFiles = [*glob.glob('**/*.html', root_dir=f'data/{epubShortFileName}', recursive=True),
                 *glob.glob('**/*.xhtml', root_dir=f'data/{epubShortFileName}', recursive=True)]
    for dir in htmlFiles:
        path = f'data/{epubShortFileName}/{dir}'
        parsedDocument = BeautifulSoup(readAll(path), 'xml')
        body = parsedDocument.find('body')
        parseTree(parsedDocument, body, body.children)
        writeAll(path, str(parsedDocument))
    newFileName = 'bold-' + epubShortFileName
    saveEpub(epubShortFileName, newFileName)
    shutil.move(f'./output/{newFileName}.zip', f'./output/{newFileName}.epub')

CoveyStephen-TheSevenHabitsOfHighlyEffectivePeople.epub
fitsdzherald-frensis-skott-velykyy-hetsbi1164.epub
Spice and Wolf, Vol. 11.epub
Spice and Wolf, Vol. 12.epub
Spice and Wolf, Vol. 13.epub
Spice and Wolf, Vol. 14.epub
Spice and Wolf, Vol. 15.epub
Spice and Wolf, Vol. 16.epub
